<a href="https://colab.research.google.com/github/pachterlab/LSCHWCP_2023/blob/main/Notebooks/Figure_3/Figure_3a/1.2_human_SARSCoV_validation_bulk_autopsy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Plot SARS-CoV counts observed in lung autopsy samples from COVID-19 patients
Data from https://www.nature.com/articles/s41467-020-20139-7

In [ ]:
!pip install -q anndata
import anndata
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.colors
%config InlineBackend.figure_format='retina'

def nd(arr):
    """
    Function to transform numpy matrix to nd array.
    """
    return np.asarray(arr).reshape(-1)

Download AnnData object from Caltech Data generated [here](https://github.com/pachterlab/LSCHWCP_2023/blob/main/Notebooks/Figure_3/Figure_3a/1.1_human_SARSCoV_validation_bulk_autopsy.ipynb).

In [ ]:
# !wget
adata = anndata.read_h5ad("human_SARSCoV_validation_bulk_autopsy.h5ad")

Load RNAish % from the original paper:

In [ ]:
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/Notebooks/Figure_3/Figure_3a/human_SARSCoV_validation_bulk_autopsy_rnaish.csv

In [ ]:
rna_ish = pd.read_csv("human_SARSCoV_validation_bulk_autopsy_rnaish.csv", sep="\t")
rna_ish["case_clean"] = rna_ish["case_clean"].values.astype(str)
rna_ish

Load virus ID to sOTU mapping:

In [ ]:
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/ID_to_taxonomy_mapping.csv
tax_df = pd.read_csv("ID_to_taxonomy_mapping.csv")
tax_df[tax_df["species"].str.contains("Severe acute respiratory syndrome")]

Plot:

In [ ]:
case_list = rna_ish["case_clean"].values

In [ ]:
fig, ax = plt.subplots(figsize=(6, 7))
fontsize = 16

target_ids = tax_df[tax_df["species"].str.contains("Severe acute respiratory syndrome-related coronavirus")]["rep_ID"].values

kb_counts = []
kb_errs = []
rnaish_counts = []
# Not including Control since there are no RNAish% values for control
for i, case in enumerate(case_list[1:]):
    kb_count_r1 = adata[(adata.obs["case"] == case) & (adata.obs["read"] == "R1"), adata.var.index.isin(target_ids)].X
    kb_count_r2 = adata[(adata.obs["case"] == case) & (adata.obs["read"] == "R2"), adata.var.index.isin(target_ids)].X

    kb_c = [kb_count_r1.sum(), kb_count_r2.sum()]
    count_mean = np.mean(kb_c)
    kb_counts.append(np.mean(kb_c))

    kb_err = [
        [count_mean - np.min(kb_c)],
        [np.max(kb_c) - count_mean],
    ]
    kb_errs.append(kb_err)

    rnaish_count = rna_ish[rna_ish["case_clean"] == case]["Viral load% by RNA ISH"].values[0]
    rnaish_counts.append(rnaish_count)

    ax.errorbar(count_mean, rnaish_count, xerr = kb_err, c= "black", ls="none", elinewidth=1)
    ax.scatter(count_mean, rnaish_count, c="#003049", edgecolors="black", s=250, zorder=2)

# ax.set_yscale("symlog")
# ax.set_xscale("symlog")
ax.set_ylabel("RNA-ISH (SARS-CoV %)", fontsize=fontsize)
ax.set_xlabel(
    "kallisto\n(raw counts for SARS-CoV)",
    fontsize=fontsize,
)

ax.text(5150, 1.5, "n=23", fontsize=fontsize)

# Add diagonal
# ax.plot([0, 1], [0, 1], transform=ax.transAxes, c="black", ls="-", lw=1, zorder=1)

ax.tick_params(axis="both", labelsize=fontsize)
ax.set_title(f"Lung autopsy samples\nfrom COVID-19 patients", fontsize=fontsize+2)

# plt.tight_layout()

ax.grid(True, which="both", color="lightgray", ls="--", lw=1)
ax.set_axisbelow(True)

plt.savefig("bulk_validation_PRJNA631753.png", dpi=300, bbox_inches="tight")

fig.show()